# Chapter 18: with, match, and else Blocks

## Context Managers and with Blocks

In [13]:
# tag::MIRROR_EX[]
import sys

class LookingGlass:

    def __enter__(self):  # <1>
        self.original_write = sys.stdout.write  # <2>
        sys.stdout.write = self.reverse_write  # <3>
        return 'JABBERWOCKY'  # <4>

    def reverse_write(self, text):  # <5>
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):  # <6>
        sys.stdout.write = self.original_write  # <7>
        if exc_type is ZeroDivisionError:  # <8>
            print('Please DO NOT divide by zero!')
            return True  # <9>
        # <10>
# end::MIRROR_EX[]

1. Python invokes `__enter__` with on argument besides `self`.
2. Hold the original `sys.stdout.write` method, so we can restore it later.
3. Monkey-patch `sys.stdout.write` with our own function.
4. Return the 'JABBERWOCKY' string just so we have something to put in the target variable `what`.
5. Our replacement to `sys.stdout.write` reverse the `text` argument and calls the original `sys.stdout.write` method.
6. Python calls `__exit__` with `None`, `None`, `None` if all went well; if an exception is raised, the three arguments get the exception data, as described after this example.
7. Restore the original `sys.stdout.write` method.
8. If the exception is not `None` and its type is `ZeroDivisionError`, print a message.
9. Return `True` to indicate that the exception was handled.
10. If `__exit__` returns `None` or any falsy value, any exception raised in the with block will be propagated.

The three argument passed to `__exit__` are:
- `exc_type`: The exception class (e.g., `ZeroDivisionError`).
- `exc_value`: The exception instance. Sometimes, parameters passed to the exception constructor--such as the error message--can be found in `exc_value.args`.
- `traceback`: A traceback object.

In [14]:
with LookingGlass() as what: #1
    print('Alice, Kitty and Snowdrop') #2
    print(what)
    print(5/0)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ
Please DO NOT divide by zero!


In [15]:
what #3

'JABBERWOCKY'

In [16]:
print('Back to normal.') #4

Back to normal.


1. The context manager is an instance of `LookingGlass`； Python calls `__enter__` on the context manager and the result is bound to `what`.
2. Print a `str`, then the value of the target variable `what`. The output of each `print` will come out reversed.
3. Now the with block is over. We can see that the value of `what` is back to normal.
4. Program output is no longer reversed.

### The contextlib Utilities

The `contextlib` package also includes:

- `closing`: A function to build context managers out of objects that provide a `close()` method but don't implement the `__enter__/__exit__` protocol.
- `suppress`: A context manager to temporarily ignore specified exceptions.
- `nullcontext`: A context manager that does nothing, to simplify conditional logic around objects that may not implement a suitable context manager. It serves as a stand-in when conditional code before the `with` block may or may not provide a context manager for the with statement.


### Using @contextmanager

The `@contextmanager` decorator is an elegant and practical tool that brings together three distinctive Python features: a function decorator, a generator, and the `with` statement.

In [17]:
import contextlib
import sys

@contextlib.contextmanager #1
def looking_glass():
    original_write = sys.stdout.write #2
    
    def reverse_write(text): #3
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write #4
    yield 'JABBERWOCKY' #5
    sys.stdout.write = original_write #6

Test-driving the `looking_glass` context manager function.

In [18]:
with looking_glass() as what: #1
    print('Alice, Kitty and Snowdrop') #2
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [19]:
what

'JABBERWOCKY'

In [20]:
print('Back to normal.')

Back to normal.


A little-known feature of `@contextmanager` is that the generators decorated with it can also be used as decorators themselves. That happens because `@contextmanager` is implemented with the `contextlib.ContextDecorator` class.

In [21]:
@looking_glass()
def verse():
    print('The time has come')    
    
verse()

emoc sah emit ehT


In [22]:
print('Back to normal.')

Back to normal.


A context manager for rewriting files in place. [inplace source code](https://www.zopatista.com/python/2013/11/26/inplace-file-rewriting/)

In [64]:
from contextlib import contextmanager
import io
import os


@contextmanager
def inplace(filename, mode='r', buffering=-1, encoding=None, errors=None,
            newline=None, backup_extension=None):
    """Allow for a file to be replaced with new content.

    yields a tuple of (readable, writable) file objects, where writable
    replaces readable.

    If an exception occurs, the old file is restored, removing the
    written data.

    mode should *not* use 'w', 'a' or '+'; only read-only-modes are supported.

    """

    # move existing file to backup, create new file with same permissions
    # borrowed extensively from the fileinput module
    if set(mode).intersection('wa+'):
        raise ValueError('Only read-only file modes can be used')

    backupfilename = filename + (backup_extension or os.extsep + 'bak')
    try:
        os.unlink(backupfilename)
    except os.error:
        pass
    os.rename(filename, backupfilename)
    readable = io.open(backupfilename, mode, buffering=buffering,
                        encoding=encoding, errors=errors, newline=newline)
    try:
        perm = os.fstat(readable.fileno()).st_mode
    except OSError:
        writable = open(filename, 'w' + mode.replace('r', ''),
                        buffering=buffering, encoding=encoding, errors=errors,
                        newline=newline)
    else:
        os_mode = os.O_CREAT | os.O_WRONLY | os.O_TRUNC
        if hasattr(os, 'O_BINARY'):
            os_mode |= os.O_BINARY
        fd = os.open(filename, os_mode, perm)
        writable = io.open(fd, "w" + mode.replace('r', ''), buffering=buffering,
                            encoding=encoding, errors=errors, newline=newline)
        try:
            if hasattr(os, 'chmod'):
                os.chmod(filename, perm)
        except OSError:
            pass
    try:
        yield readable, writable
    except Exception:
        # move backup back
        try:
            os.unlink(filename)
        except os.error:
            pass
        os.rename(backupfilename, filename)
        raise
    finally:
        readable.close()
        writable.close()
        try:
            os.unlink(backupfilename)
        except os.error:
            pass

In [ ]:
# need to run in linux os.
import csv

with inplace('./test.txt', 'r', newline='') as (infh, outfn):
    reader = csv.reader(infh)
    writer = csv.writer(outfn)
    
    for row in reader:
        row += ['new', 'columns']
        writer.writerow(row)

## Pattern Matching in lis.py: A Case Study

### Scheme Syntax

Greatest common divisor in Scheme

```
(define (mod m n)
    (- m (* (quotient m n))))

(define (gcd m n)
    (if (= n 0)
        m
        (gcd n (mod m n))))

(display (gcd 18 45))
```

In [ ]:
# Same as the example above, written in Python

def mod(m, n):
    return m - (m // n * n)

def gcd(m, n):
    return m if n == 0 else gcd(n, mod(m, n))

print(gcd(18, 45))

### The Parser

In [25]:
from lis import parse

parse('1.5')

1.5

In [27]:
parse('ni!')

'ni!'

In [28]:
parse('(gcd 18 45)')

['gcd', 18, 45]

In [29]:
parse('''
(define double
  (lambda (n)
    (* 2 n)))
''')

['define', 'double', ['lambda', ['n'], ['*', 2, 'n']]]

In [30]:
parse('(lambda (a b) (* (/ a b) 100))')

['lambda', ['a', 'b'], ['*', ['/', 'a', 'b'], 100]]

### The Environment

In [31]:
from lis import Environment

inner_env = {'a': 2}
outer_env = {'a': 0, 'b': 1}
env = Environment(inner_env, outer_env)
env

Environment({'a': 2}, {'a': 0, 'b': 1})

In [32]:
print(env['a']) #1
print(env['b'])

2
1


In [33]:
env['a'] = 111 #2
env['c'] = 222
env

Environment({'a': 111, 'c': 222}, {'a': 0, 'b': 1})

In [34]:
env.change('b', 333) #3
print(env['a'])
print(env['b'])
print(env['c'])

111
333
222


1. When reading values, `Environment` works as `ChainMap`: keys are searched in the nested mappings from left to right. That's why the value of `a` in the `outer_env` is shadowed by the value of `a` in the `inner_env`.
2. Assigning with [] overwrites or inserts new items, but always in the first mapping, `inner_env` in this example.
3. `env.change('b', 333)` seeks the `b` key in the nested mappings and changes its value in the first mapping where it is found.

`Standard_env()` builds and returns the global environment. It's similar to Python's  `__builtins__` module that is always available. The `env` mapping is loaded with:

- All functions from Python's `math` module
- Selected operators from Python's `operate` module
- Simple but powerful functions built Python's `lambda`
- Python built-ins renamed, like `callable` as `procedure?`, or directly mapped, like `round`

In [35]:
from lis import standard_env

env = standard_env()

# 96. why so many entries here?
# Seems come from math module and operator module: math.pyi and operator.py
print(len(env.maps[0])) 

env.maps

98


[{'__name__': 'math',
  '__doc__': 'This module provides access to the mathematical functions\ndefined by the C standard.',
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(name='math', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'),
  'acos': <function math.acos(x, /)>,
  'acosh': <function math.acosh(x, /)>,
  'asin': <function math.asin(x, /)>,
  'asinh': <function math.asinh(x, /)>,
  'atan': <function math.atan(x, /)>,
  'atan2': <function math.atan2(y, x, /)>,
  'atanh': <function math.atanh(x, /)>,
  'ceil': <function math.ceil(x, /)>,
  'copysign': <function math.copysign(x, y, /)>,
  'cos': <function math.cos(x, /)>,
  'cosh': <function math.cosh(x, /)>,
  'degrees': <function math.degrees(x, /)>,
  'dist': <function math.dist(p, q, /)>,
  'erf': <function math.erf(x, /)>,
  'erfc': <function math.erfc(x, /)>,
  'exp': <function math.exp(x, /)>,
  'expm1': <function math.expm1(x, /)>,
  'fabs': <function m

In [36]:
print(env.maps[0]['+'])
print(env.maps[0]['//'])
print(env.maps[0]['%'])
print(env.maps[0]['pi'])
print(env.maps[0]['sin'])
print(env.maps[0]['max'])
print(env.maps[0]['gcd'])

<built-in function add>
<built-in function floordiv>
<built-in function mod>
3.141592653589793
<built-in function sin>
<built-in function max>
<built-in function gcd>


### The REPL

- `repl(prompt: str = 'lis.py> ') -> NoReturn`:  Calls `standard_env()` to provide built-in functions for the global environment, then enters an infinite loop, reading and parsing each input line, evaluating it in the global environment, and displaying the result--unless it's `None`. The `global_env` may be modified by `evaluate`. For example, when a user defines a new global variable or named function, it is stored in the first mapping of the environment--the empty `dict` in the `Environment` constructor call in the first line of `repl`.
- `lispstr(exp: object) -> str`:  The inverse function of `parse`, given a Python object representing an expression, `parse` returns the Scheme source code for it. For example, given `['+', 2, 3]`, the result is `(+ 2 3)`.

### The Evaluator

The `evaluate` function takes an `Expression` built by `parse` and an `Environment`.

The body of `evaluate` is a single `match` statement with an expression `exp` as the subject. The `case` patterns express the syntax and semantics of Scheme with amazing clarity.

In [37]:
from lis import parse, evaluate, standard_env
evaluate(parse('1.5'), {})

1.5

In [38]:
evaluate(parse('+'), standard_env())

<function _operator.add(a, b, /)>

In [39]:
evaluate(parse('(quote no-such-thing)'), standard_env())

'no-such-thing'

In [40]:
evaluate(parse('(quote (99 bottles of beer))'), standard_env())

[99, 'bottles', 'of', 'beer']

In [41]:
evaluate(parse('(quote (/ 10 0))'), standard_env())

['/', 10, 0]

In [42]:
evaluate(parse('(if (= 3 3) 1 0)'), standard_env())

1

In [43]:
evaluate(parse('(if (= 3 4) 1 0)'), standard_env())

0

In [44]:
expr = '(lambda (a b) (* (/ a b) 100))'
print(parse(expr))
f = evaluate(parse(expr), standard_env())
f

['lambda', ['a', 'b'], ['*', ['/', 'a', 'b'], 100]]


In [45]:
f(15, 20)

75.0

In [46]:
global_env = standard_env()
evaluate(parse('(define answer (* 7 6))'), global_env)
global_env['answer']

42

In [47]:
len(global_env.maps[0])

99

In [48]:
evaluate(parse('answer'), global_env)

42

In [54]:
percent = '(define (perc a b) (* (/ a b) 100))'
evaluate(parse(percent), global_env)
global_env['perc']

In [55]:
global_env['perc'](170, 200)

85.0

The use of the `set!` form is related to the use of the `nonlocal` keyword in Python: declaring `nonlocal x` allow `x = 10` to update a previously defined `x` variable outside of local scope.

In [51]:
from lis import parse, evaluate, standard_env

expr = '''
(define (make-averager)
    (define count 0)
    (define total 0)
    (lambda (new-value)
        (set! count (+ count 1))
        (set! total (+ total new-value))
        (/ total count)
    )
)
'''

evaluate(parse(expr), global_env)
global_env['make-averager']

In [52]:
avg = '(define avg (make-averager))'
evaluate(parse(avg), global_env)
global_env['avg']

In [53]:
print(global_env['avg'](10))
print(global_env['avg'](11))
print(global_env['avg'](15))

10.0
10.5
12.0


In [56]:
evaluate(parse('(perc (* 12 14) (- 500 100))'), global_env)

42.0

### Procedure: A Class Implementing a Closure

The `Procedure` class could very well be named `Closure`, because that's what it represents: a function together with an environment. The function definition includes the name of the parameters and the expressions that make up the body fo the function. The environment is used when the function is called to provide the values of the `free variables`: variables that appear in the body of the function but are not parameters, local variables, or global variables.

### Using OR-patterns

```python
case int(x) | float(x):
    return x
```

In the context of a `case` clause, the | operator has a special meaning. It does not trigger the `__or__` special method, which handles expressions like `a | b` in Python, where it is overloaded to perform operations such as set union or integer bitwise-or, depending on the operands.

## Do this, Then That: else Blocks Beyond if

The `else` clause can be used not only in `if` statements, but also in `while`, `for`, and `try` statements. 

Here are the rules for using `else` with these statements:

- `for`: The `else` block will run only `if` and when `for` loop runs to completion (i.e., not when the loop is terminated by a `break` statement).
- `while`: The `else` block will run only `if` and when the `while` loop exits because the condition became falsy (i.e., not when the loop is terminated by a `break` statement).
- `try`: The `else` block will only run `if` no exception is raised in the `try` block. The office docs also state: "Exceptions in the else clause are not handled by the preceding except clauses."
- In all cases, the `else` clause is also skipped if an exception or a `return`, `break`, or `continue` statement causes control to jump out of the main block of the compound statement.

In [60]:
my_list = ['apple', 'banana', 'grapes', 'pear']

for item in my_list:
    if item == 'banana':
        break
else:
    raise ValueError('No banana flavor found!')

EAFP

Easier to ask for forgiveness than permission. This common Python coding style assumes the existence of valid keys or attributes and catches exceptions if the assumption proves false. This clean and fast style is characterized by the presence of many `try` and `except` statements. The technique contrasts with the LBYL style common to many other languages such as C.

LBYL

Look before you leap. This coding style explicitly tests for pre-conditions before making calls or lookups. This style contrasts with the EAFP approach and is characterized by the presence of many `if` statements. In a multi-threaded, the LBYL approach can risk introducing a race condition between "the looping" and "the leaping." For example, the code, if key in mapping: return mapping[key] can fail if another thread removes key from mapping after the test, but before the lookup. The issue can be solved with locks or by using the EAFP approach.